In [1]:
import numpy as np
import tensorflow 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
structures = np.loadtxt("structures_120k.txt")

In [3]:
structures.shape

(120000, 8)

In [4]:
structures = structures/200

In [5]:
structures[:5]

array([[0.935, 0.52 , 0.52 , 0.84 , 0.795, 0.86 , 0.575, 0.505],
       [0.995, 0.85 , 0.59 , 0.125, 0.365, 0.695, 0.99 , 0.21 ],
       [0.665, 0.205, 0.82 , 0.485, 0.44 , 0.315, 0.63 , 0.185],
       [0.515, 0.835, 0.6  , 0.725, 0.585, 0.795, 0.53 , 0.58 ],
       [0.255, 0.885, 0.565, 0.27 , 0.4  , 0.87 , 0.825, 0.725]])

In [6]:
spectra = np.loadtxt("spectra_120k.dat")

In [7]:
spectra = spectra.reshape(120000,61,4)

In [8]:
spectra.shape

(120000, 61, 4)

In [9]:
structures_train,structures_test,spec_train,spec_test = structures[:110000],structures[110000:],spectra[:110000],spectra[110000:]

In [10]:
spec_test.shape

(10000, 61, 4)

In [11]:
'''import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans


# Compute WCSS for different values of k (from 1 to 1200)
wcss = []
for k in range(1, 700):  # Test for K from 1 to 1200 clusters
    mb_kmeans = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=1000)  # Adjust batch_size as needed
    mb_kmeans.fit(structures_train)
    wcss.append(mb_kmeans.inertia_)
# Plot the Elbow graph
plt.plot(range(1, 700), wcss)
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of clusters (K)')
plt.ylabel('WCSS (Inertia)')
plt.show()'''


"import matplotlib.pyplot as plt\nfrom sklearn.cluster import KMeans\nfrom sklearn.cluster import MiniBatchKMeans\n\n\n# Compute WCSS for different values of k (from 1 to 1200)\nwcss = []\nfor k in range(1, 700):  # Test for K from 1 to 1200 clusters\n    mb_kmeans = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=1000)  # Adjust batch_size as needed\n    mb_kmeans.fit(structures_train)\n    wcss.append(mb_kmeans.inertia_)\n# Plot the Elbow graph\nplt.plot(range(1, 700), wcss)\nplt.title('Elbow Method for Optimal K')\nplt.xlabel('Number of clusters (K)')\nplt.ylabel('WCSS (Inertia)')\nplt.show()"

In [12]:
k = 500 # You can experiment with this number
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(structures_train)

KMeans(n_clusters=500, random_state=42)

In [13]:
# Get the labels (which cluster each point belongs to)
labels = kmeans.labels_

In [14]:
# You can also get the cluster centers if you want to visualize them
centroids = kmeans.cluster_centers_

In [15]:
centroids.shape

(500, 8)

In [16]:
from collections import Counter

# Step 2: Select equal samples from each cluster
samples_per_cluster = 80  # Number of samples you want to select from each cluster (adjust as needed)

# Initialize lists to hold the selected data
selected_inputs = []
selected_outputs = []

# Iterate through each cluster
for cluster_id in range(k):
    # Find all samples that belong to the current cluster
    cluster_indices = np.where(labels == cluster_id)[0]
    
    # Shuffle the indices (just in case they're sorted or grouped in a non-random way)
    np.random.shuffle(cluster_indices)
    
    # Select the first `samples_per_cluster` samples from this cluster
    selected_indices = cluster_indices[:samples_per_cluster]
    
    # Append the corresponding input and output data for these samples
    selected_inputs.append(structures_train[selected_indices])   # Corresponding inputs
    selected_outputs.append(spec_train[selected_indices])  # Corresponding outputs

# Convert the lists into arrays (for easier processing in the neural network)
selected_inputs = np.vstack(selected_inputs)
selected_outputs = np.vstack(selected_outputs)

# Check the shape of the selected data
print(selected_inputs.shape)   # Should be (samples_per_cluster * k, 8)
print(selected_outputs.shape)  # Should be (samples_per_cluster * k, 61, 4)


(40000, 8)
(40000, 61, 4)


In [17]:
#k = 500  #here k is no clusters
#n_samples = 5000  #no. of data points to select

In [18]:
#kmeans = KMeans(n_clusters=k)
#kmeans.fit(structures_train)

In [19]:
#labels = kmeans.labels_

In [20]:
#samples_per_cluster = n_samples // k

In [21]:
#sampled_structures = []
#sampled_spectra = []

In [22]:
'''for cluster in range(k):
    # Get the indices of data points in this cluster
    cluster_indices = np.where(labels == cluster)[0]
    
    # Randomly sample 'samples_per_cluster' from this cluster (without fixed seed)
    sampled_indices = np.random.choice(cluster_indices, samples_per_cluster, replace=False)
    
    # Append the sampled data points and their corresponding output spectra
    sampled_structures.append(structures_train[sampled_indices])
    sampled_spectra.append(spec_train[sampled_indices])'''

"for cluster in range(k):\n    # Get the indices of data points in this cluster\n    cluster_indices = np.where(labels == cluster)[0]\n    \n    # Randomly sample 'samples_per_cluster' from this cluster (without fixed seed)\n    sampled_indices = np.random.choice(cluster_indices, samples_per_cluster, replace=False)\n    \n    # Append the sampled data points and their corresponding output spectra\n    sampled_structures.append(structures_train[sampled_indices])\n    sampled_spectra.append(spec_train[sampled_indices])"

In [23]:
'''sampled_structures = np.vstack(sampled_structures)
sampled_spectra = np.vstack(sampled_spectra)'''

'sampled_structures = np.vstack(sampled_structures)\nsampled_spectra = np.vstack(sampled_spectra)'

In [24]:
#sampled_spectra.shape

In [25]:
model = tensorflow.keras.Sequential([
    tensorflow.keras.layers.Input(shape=(8,)),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Reshape((16, 8)),
    tensorflow.keras.layers.Conv1D(128, kernel_size=5, padding='same', activation='elu'),

    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(256, activation='elu'),

    tensorflow.keras.layers.Reshape((32, 8)),
    tensorflow.keras.layers.Conv1D(256, kernel_size=5, padding='same', activation='elu'),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Reshape((16, 8)),
    tensorflow.keras.layers.Conv1D(128, kernel_size=5, padding='same', activation='elu'),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Reshape((16, 8)),
    tensorflow.keras.layers.Conv1D(128, kernel_size=5, padding='same', activation='elu'),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Reshape((16, 8)),
    tensorflow.keras.layers.Conv1D(128, kernel_size=5, padding='same', activation='elu'),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Reshape((16, 8)),
    tensorflow.keras.layers.Conv1D(128, kernel_size=5, padding='same', activation='elu'),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Dense(128, activation='elu'),
    tensorflow.keras.layers.Dense(244, activation='sigmoid'),
    tensorflow.keras.layers.Reshape((61, 4)),
])

In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 16, 128)        │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 32, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 256)        │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 16, 128)        │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 16, 128)        │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_4 (Reshape)             │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 16, 128)        │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_5 (Reshape)             │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 16, 128)        │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       262,27

 Total params: 2,708,212 (10.33 MB)

 Trainable params: 2,708,212 (10.33 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.001),loss='mse')

In [28]:
#history =  model.fit(sampled_structures,sampled_spectra,epochs=100,batch_size=256,validation_split=0.3)
history =  model.fit(selected_inputs,selected_outputs,epochs=100,batch_size=256,validation_split=0.3)


Epoch 1/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - loss: 0.0527 - val_loss: 0.0243
Epoch 2/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - loss: 0.0243 - val_loss: 0.0230
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - loss: 0.0221 - val_loss: 0.0198
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - loss: 0.0195 - val_loss: 0.0187
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 0.0182 - val_loss: 0.0180
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - loss: 0.0172 - val_loss: 0.0164
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - loss: 0.0164 - val_loss: 0.0165
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.0162 - val_loss: 0.0156
Epoch 9/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.0157 - val_loss: 0.0154
Epoch 10/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.0154 - val_loss: 0.0148
Epoch 11/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - loss: 0.0148 - val_loss: 0.0139
Epoch 12/100
110/110 ━━━━━━━━━

In [29]:
predicted_spectra_train = model.predict(structures_train)


3438/3438 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step


In [30]:
mse = tensorflow.keras.losses.MeanSquaredError()
loss = mse(spec_train, predicted_spectra_train).numpy()

print(f"The loss for 110k data of test1 is  {loss}")

The loss for 110k data of test1 is  0.0012161755003035069


In [31]:
predicted_spectra_test = model.predict(structures_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [32]:
mne = tensorflow.keras.losses.MeanSquaredError()
losses = mse(spec_test,predicted_spectra_test).numpy()

print(f"The loss for 10k data of test1 is  {losses}")

The loss for 10k data of test1 is  0.001333255204372108
